# Setup

In [ ]:
username = 'MarcelloCeresini'
repository = 'QuestionAnswering'

# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !pip3 install transformers
    !apt install libomp-dev
    !pip install faiss
    !git clone https://www.github.com/{username}/{repository}.git
    #from google.colab import drive
    #drive.mount('/content/drive/')
    %cd /content/QuestionAnswering/src
    using_TPU = True    # If we are running this notebook on Colab, use a TPU
    # Google cloud credentials
    %env GOOGLE_APPLICATION_CREDENTIALS=/content/drive/MyDrive/Uni/Magistrale/NLP/Project/nlp-project-338723-0510aa0a4912.json
except:
    IN_COLAB = False
    using_TPU = False   # If you're not on Colab you probably won't have access to a TPU

# Imports

In [2]:
import faiss
import numpy as np
import os

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')
    datasets_dir = '/content/drive/MyDrive/Uni/Magistrale/NLP/Project/datasets/dpr/'
else:
    # Create the folder where we'll save the weights of the model
    datasets_dir = os.path.join("data", "training_dpr", "dataset")

representations_dir = os.path.join(datasets_dir, 'representations')

os.makedirs(datasets_dir, exist_ok=True)
os.makedirs(representations_dir, exist_ok=True)

# Indices creation

Open matrices using memmap.

In [4]:
train_matrix = np.memmap(os.path.join(representations_dir, 'train_paragraphs_encodings.npy'), 
                         mode='r', dtype=np.float32, shape=(13975,512,768))[:,0,:]

val_matrix = np.memmap(os.path.join(representations_dir, 'val_paragraphs_encodings.npy'), 
                        mode='r', dtype=np.float32, shape=(4921,512,768))[:,0,:]

test_matrix = np.memmap(os.path.join(representations_dir, 'test_paragraphs_encodings.npy'), 
                        mode='r', dtype=np.float32, shape=(2067,512,768))[:,0,:]

train_matrix.shape, val_matrix.shape, test_matrix.shape

((13975, 768), (4921, 768), (2067, 768))

## Test

Create test indices

In [5]:
index_test = faiss.IndexHNSWFlat(768,512)
index_test.add(np.ascontiguousarray(test_matrix))
faiss.write_index(index_test, os.path.join(datasets_dir, 'indices/test_index'))
del index_test, test_matrix

## Val

Create validation indices

In [6]:
index_val = faiss.IndexHNSWFlat(768,512)
index_val.add(np.ascontiguousarray(val_matrix))
faiss.write_index(index_val, os.path.join(datasets_dir, 'indices/val_index'))
del index_val, val_matrix

## Train (heaviest index)

Create training indices

In [ ]:
index_train = faiss.IndexHNSWFlat(768,512)
index_train.add(np.ascontiguousarray(train_matrix))
faiss.write_index(index_train, 'train_index')
del index_train, train_matrix 